# BERT 토큰 분류 실습 - 개체명 인식 (Named Entity Recognition, NER)
![](https://raw.githubusercontent.com/nbroad1881/kaggle-images/main/feedback-prize-effectiveness/token-classification-diagram.png)
### 요구사항
1. token classification loss 구현하기
2. token classification metric 구현하기



In [ ]:
!pip install -q transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.0 MB/s eta 0:00:00


In [ ]:
from pprint import pprint

## 데이터셋 살펴보기

In [ ]:
from datasets import load_dataset

wnut = load_dataset("wnut_17")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [ ]:
# 문장이 word 단위로 쪼개져서 tokens에 list로 들어가있는 상태입니다.
# 이후 각 word마다 tag가 부여되어 있습니다.
# 하지만 우리는 tokenizer로 word를 subword로 분리해야 합니다 (예: @paulwalk -> @ ##paul ##walk로 분리될 수 있음)
# 그래서 전처리를 추가로 수행해 보겠습니다
pprint(wnut["train"][0])

{'id': '0',
 'ner_tags': [0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              7,
              8,
              8,
              0,
              7,
              0,
              0,
              0,
              0,
              0,
              0,
              0,
              0],
 'tokens': ['@paulwalk',
            'It',
            "'s",
            'the',
            'view',
            'from',
            'where',
            'I',
            "'m",
            'living',
            'for',
            'two',
            'weeks',
            '.',
            'Empire',
            'State',
            'Building',
            '=',
            'ESB',
            '.',
            'Pretty',
            'bad',
            'storm',
            'here',
            'last',
            'evening',
         

In [ ]:
# 각각의 정수 index가 어떤 class인지는 feature 필드에 저장되어 있습니다.
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

ner tag는 각 word의 entity를 담고 있습니다. 하지만 B와 I라는 prefix가 있는데요.

- B: entity의 시작을 의미합니다.<br/>
- I: 이전 entity의 내부임을 의미합니다. 예를 들어, "Empire State Building" 이 띄어쓰기로 세 토큰으로 나뉜다고 했을 때, 이 세 토큰은 location에 해당합니다. 하지만 Empire는 B-location, State는 I-location이 됩니다. .<br/>
- 0: 어떤 entity에도 속하지 않는 것을 의미합니다. <br/>


전처리 과정 seudo code
```
tokens = ["Empire", "State", "Building"]
ner_tags = [7, 8, 8]

# 토크나이저가 아래처럼 subword로 쪼갰다고 가정해 봅시다.
tokenized = tokenizer(tokens) # ["Emp", "##ire", "State", "Build", "##ing"]

# 쪼개진 토큰이 원래 몇번째 word인지 알아낸 다음
words_ids = [0, 0, 1, 2, 2]

# ner tag를 쪼개진 토큰에 맞춰서 5개의 라벨로 늘려야 됩니다.

# 각 토큰을 13개의 ner tag 중 하나로 분류할 것이기 때문에, 라벨을 정수 인덱스로 바꿔주어야합니다.
labels = [7, 7, 8, 8, 8]

# 하지만 subword가 많아지면 특정 class가 너무 길게 이어질 수 있습니다. 그러면 학습 불균형과 오버피팅을 초래할 수 있습니다.
# 따라서 한 word에서 첫번째 subword를 제외한 나머지는 -100으로 처리해서 loss에서 제외시켜 불균형을 막아야 합니다.
labels = [7, -100, 8, 8, -100]
```

In [ ]:
# bert-tiny가 작고 빨라요, 더 크고 뛰어난 모델을 쓰고 싶다면, 다른 사전학습 모델을 찾아 넣어봅시다.

# model_id = "distilbert-base-uncased"
model_id = "prajjwal1/bert-tiny"

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
example = wnut["train"][0]

# is_split_into_words=True 는 문자열이 아니라 이미 토큰으로 나눠진 list로 들어간다는 것을 의미합니다.
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)

# BERT model에 따라 token_type_ids가 없는 경우가 있습니다.
pprint(tokenized_input)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
pprint(tokens)

{'attention_mask': [1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1,
                    1],
 'input_ids': [101,
               1030,
               2703,
               17122,
               2009,
               1005,
               1055,
               1996,
               3193,
               2013,
       

In [ ]:
# word_ids는 각 토큰이 몇 번째 word인지를 의미합니다. CLS, SEP 같은 special token은 None으로 들어가 있습니다.
tokenized_input.word_ids(0)

[None,
 0,
 0,
 0,
 1,
 2,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 None]

쪼개진 토큰에 맞춰 라벨을 늘려봅시다. 또 special token은 label로 -100을 지정해서 학습에서 제외되도록 합시다.

In [ ]:
# 빠른 처리를 위해 여러개를 한번에 하겠습니다(batch 처리)
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        # 원래 word id 가져오기
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # 구현해봅시다
            # 1. special tokens은 -100.
            # 2. subword로 분리될 경우 첫번째 토큰을 제외하고 -100
            ########
            ########

            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


test_batch = tokenize_and_align_labels(
    wnut["train"][0:4]
)
print(test_batch)


# 아래 값과 일치해야 정답
answer = {'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], [101, 2013, 2665, 2739, 7959, 2098, 1024, 6289, 7011, 8908, 15117, 2005, 10878, 2400, 2000, 13292, 1012, 1019, 8299, 1024, 1013, 1013, 4714, 3126, 2140, 1012, 4012, 1013, 2484, 8490, 3501, 22025, 102], [101, 1052, 20959, 2229, 2327, 2753, 5855, 5049, 4620, 1997, 2257, 2230, 1012, 1012, 1012, 8299, 1024, 1013, 1013, 2978, 1012, 1048, 2100, 1013, 1038, 18195, 2100, 2480, 2692, 1001, 5855, 102], [101, 2651, 2003, 2026, 2197, 2154, 2012, 1996, 2436, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, -100, -100, 0, 0, -100, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 7, 8, 8, 0, 7, -100, 0, 0, 0, 0, 0, 0, 0, 0, -100], [-100, 0, 0, 0, -100, -100, 0, 5, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], [-100, 1, -100, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, 0, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 0, -100, -100], [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

def assert_list(a, b):
  assert len(a) == len(b)
  for aa, bb in zip(a, b):
    assert len(aa) == len(bb)
    assert all(x == y for x, y in zip(aa, bb))

for k, v in answer.items():
  assert_list(v, test_batch[k])

print("pass!")

{'input_ids': [[101, 1030, 2703, 17122, 2009, 1005, 1055, 1996, 3193, 2013, 2073, 1045, 1005, 1049, 2542, 2005, 2048, 3134, 1012, 3400, 2110, 2311, 1027, 9686, 2497, 1012, 3492, 2919, 4040, 2182, 2197, 3944, 1012, 102], [101, 2013, 2665, 2739, 7959, 2098, 1024, 6289, 7011, 8908, 15117, 2005, 10878, 2400, 2000, 13292, 1012, 1019, 8299, 1024, 1013, 1013, 4714, 3126, 2140, 1012, 4012, 1013, 2484, 8490, 3501, 22025, 102], [101, 1052, 20959, 2229, 2327, 2753, 5855, 5049, 4620, 1997, 2257, 2230, 1012, 1012, 1012, 8299, 1024, 1013, 1013, 2978, 1012, 1048, 2100, 1013, 1038, 18195, 2100, 2480, 2692, 1001, 5855, 102], [101, 2651, 2003, 2026, 2197, 2154, 2012, 1996, 2436, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3394 [00:00<?, ? examples/s]

Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

In [ ]:
tokenized_wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'input_ids': [101,
  1030,
  2703,
  17122,
  2009,
  1005,
  1055,
  1996,
  3193,
  2013,
  2073,
  1045,
  1005,
  1049,
  2542,
  2005,
  2048,
  3134,
  1012,
  3400,
  2110,
  2311,
  1027,
  9686,
  2497,
  1012,
  3492,
  2919,
  4040,
  2182,
  2197,
  3944,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  -100,
  -100,
  0,
  0,
  -100,
  0,
  0,
  0,
  0,
  0,
  0,
  

Collator는 배치 크기를 통일시키고 padding을 추가해줍니다. label의 경우는 padding으로 -100이 추가됩니다.

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True, return_tensors="pt")

In [ ]:
# "token_type_ids"는 BERT 모델에 따라 없는 경우도 있습니다.
# 필요한 BERT 모델의 경우에도 따로 추가해주지 않으면 huggingface model은 자체적으로 0으로 넣습니다.
# 이번 예제는 token_type_ids가 전부 0이므로 별도로 추가하지 않겠습니다.

allowed_keys = ["input_ids", "attention_mask", "labels"]
def filter_keys(d):
  return {k:v for k,v in d.items() if k in allowed_keys}

batch = [filter_keys(tokenized_wnut["train"][i]) for i in range(4)]
batch = data_collator(batch)
pprint(batch)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[  101,  1030,  2703, 17122,  2009,  1005,  1055,  1996,  3193,  2013,
          2073,  1045,  1005,  1049,  2542,  2005,  2048,  3134,  1012,  3400,
          2110,  2311,  1027,  9686,  2497,  1012,  3492,  2919,  4040,  2182,
          2197,  3944,  1012,   102],
        [  101,  2013,  2665,  2739,  7959,  2098,  1024,  6289,  7011,  8908,
         15117,  2005, 10878,  2400,  2000, 13292,  1012,  1019,  8299,  1024,
          1013,  1013,  4714,  3126,  2140,  101

## 모델 설계

In [ ]:
from transformers import AutoModel

# 사전학습된 모델 불러오기
model = AutoModel.from_pretrained(model_id)
model

DistilBertModel(
  (embeddings): Embeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): Transformer(
    (layer): ModuleList(
      (0-5): 6 x TransformerBlock(
        (attention): MultiHeadSelfAttention(
          (dropout): Dropout(p=0.1, inplace=False)
          (q_lin): Linear(in_features=768, out_features=768, bias=True)
          (k_lin): Linear(in_features=768, out_features=768, bias=True)
          (v_lin): Linear(in_features=768, out_features=768, bias=True)
          (out_lin): Linear(in_features=768, out_features=768, bias=True)
        )
        (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (ffn): FFN(
          (dropout): Dropout(p=0.1, inplace=False)
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Li

In [ ]:
# model의 출력값 확인 및 last_hidden_state 얻어오기
batch_no_labels = {k: v for k, v in batch.items() if k != "labels"}
model_output = model(**batch_no_labels)
pprint(model_output)
print(model_output.last_hidden_state.shape)

BaseModelOutput(last_hidden_state=tensor([[[ 0.0892, -0.0971,  0.1402,  ..., -0.2458,  0.2778,  0.5063],
         [ 0.6237, -0.0293,  0.3670,  ..., -0.0463,  0.2964,  0.3158],
         [ 0.3887,  0.0314,  0.3661,  ..., -0.0778,  0.1596,  0.0641],
         ...,
         [-0.1613, -0.5747,  0.5935,  ..., -0.2910, -0.0349, -0.6906],
         [-0.1691, -1.0051,  0.0483,  ...,  0.2551,  0.0237, -0.1812],
         [ 0.8026,  0.4936, -0.1386,  ..., -0.1136, -0.4376, -0.2944]],

        [[-0.1844, -0.2878, -0.0354,  ..., -0.0800,  0.4346,  0.3636],
         [ 0.1681, -0.0698,  0.1989,  ...,  0.1692,  0.5744,  0.3007],
         [-0.0233, -0.1929,  0.4191,  ..., -0.0058,  0.1676,  0.0071],
         ...,
         [-0.0954, -0.1214,  0.3115,  ..., -0.4472, -0.0671, -0.1784],
         [ 0.7238,  0.1994, -0.0797,  ...,  0.2798, -0.5293, -0.3254],
         [-0.0400, -0.0989,  0.3018,  ..., -0.0646,  0.1012,  0.0380]],

        [[ 0.0433, -0.2138,  0.2123,  ..., -0.0848,  0.2782,  0.2532],
         [ 

### Loss 계산
1. 각 토큰의 hidden state를 13개의 카테고리로 바꾼다 (nn.Linear)
2. cross-entropy를 계산한다
3. label이 -100인 값은 loss 계산에서 제외한다

In [ ]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}
num_labels = 13

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 모델의 hidden state 크기는 config에 있다.
model_hidden_size = model.config.hidden_size

X = model(input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]).last_hidden_state
X = nn.Linear(model_hidden_size, num_labels, bias=False)(X)
print(X)
print(X.shape)

tensor([[[-0.2348,  0.2882, -0.0864,  ...,  0.2738, -0.5125,  0.1661],
         [-0.1283,  0.2787,  0.3090,  ...,  0.2579, -0.0360, -0.0396],
         [-0.0628,  0.3574, -0.0612,  ..., -0.1146, -0.1115,  0.0206],
         ...,
         [-0.0156,  0.1745, -0.1095,  ...,  0.0178,  0.2308, -0.0874],
         [-0.2373,  0.0463,  0.1326,  ...,  0.0990,  0.0318,  0.1807],
         [-0.2678, -0.0423,  0.0473,  ...,  0.4928,  0.1616, -0.3987]],

        [[-0.4577,  0.2801,  0.0121,  ...,  0.2514, -0.4057,  0.0569],
         [-0.2592, -0.0150,  0.1872,  ...,  0.1072, -0.0913, -0.1847],
         [-0.2033,  0.2556,  0.0863,  ..., -0.0394,  0.1865, -0.0378],
         ...,
         [-0.2653,  0.2387, -0.0469,  ...,  0.0063,  0.0135, -0.3575],
         [-0.4064, -0.2306,  0.1955,  ...,  0.4315,  0.2911, -0.5788],
         [-0.1420,  0.2432, -0.1035,  ..., -0.0196, -0.0411, -0.0013]],

        [[-0.3556,  0.1784, -0.0399,  ...,  0.2105, -0.4978,  0.0990],
         [-0.3251,  0.2589,  0.0282,  ...,  0

### loss 계산하는 함수를 직접 구현해보자

```
# 아래와 같은 logits, labels 있다고 했을 때
logits = [
  [ 0.1, 0.5, 0.4],
  [ 0.1, 0.4, 0.5],
  [ 0.5, 0.2, 0.3],
]
labels = [0, 2, -100]

# 최종 loss는 다음과 같아야 한다 (3번째는 label이 -100이라서 무시)
loss = sum(log(0.1) + log(0.5) + 0) / 2
```

계산 방법
1. `torch.gather`를 이용해서, 정답 index의 값을 빼온다
1. log를 취해서 loglikelihood를 계산한다
1. labels가 -100인 값들을 제외하고 평균을 취한다: masked_fill을 이용해서, -100이면 0, 아니면 1인 mask 텐서를 만든다. 이후 loglikelihood에 곱해서 0으로 바꾸면 된다.




In [ ]:
# torch.gather & masked_fill 예시,

logits = torch.tensor([
  [ 0.1, 0.5, 0.4],
  [ 0.1, 0.4, 0.5],
  [ 0.5, 0.2, 0.3],
])
labels = torch.tensor([0, 2, -100])

# -100 인 label은 gather할 경우 에러가 난다. 따라서 0으로 바꿔준다 - 아무 값으로 바꿔도 상관없다. 나중에 loss 합칠 때 제외하면 된다
labels_nonzero = labels.masked_fill(labels < 0, 0)

print("labels_nonzero", labels_nonzero)
gathered_logits = torch.gather(logits, -1, labels_nonzero.unsqueeze(-1))
print("gathered_logits", gathered_logits)

labels_mask = (labels >= 0).float()
print("labels_mask", labels_mask)
print("-100인 label은 log loss에서 0으로 만들기", gathered_logits.log().squeeze(-1) * labels_mask)

labels_nonzero tensor([0, 2, 0])
gathered_logits tensor([[0.1000],
        [0.5000],
        [0.5000]])
labels_mask tensor([1., 1., 0.])
-100인 label은 log loss에서 0으로 만들기 tensor([-2.3026, -0.6931, -0.0000])


In [ ]:
# cross-entropy 계산
def token_classification_loss(logits, labels):
  # logits: shape[batch, seq, labels]
  # labels: shape[batch, seq]

  logits = logits.softmax(-1)


  ######## 구현해봅시당~~~~


  loss = ??
  return loss

# 2.6628 정답
token_classification_loss(X, batch["labels"])

tensor(2.6880, grad_fn=<DivBackward0>)

model과 classification head를 합쳐서 하나의 모듈로 만들어봅시다

In [ ]:
from typing import Optional

class TokenClassificationModule(nn.Module):
  def __init__(self, pretrained_model_id: str, num_classes: int):
    super().__init__()

    self.model = AutoModel.from_pretrained(pretrained_model_id)
    self.class_head = nn.Linear(self.model.config.hidden_size, num_classes, bias=False)

  def forward(self, input_ids: torch.Tensor, attention_mask: torch.Tensor):
    X = self.model(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
    X = self.class_head(X)
    return X

model = TokenClassificationModule(model_id, num_labels)
model(**batch_no_labels)

tensor([[[ 0.1958, -0.1832, -0.2471,  ...,  0.2892, -0.1137, -0.1245],
         [ 0.0852, -0.2177, -0.1026,  ..., -0.2209,  0.1245, -0.1567],
         [-0.1381,  0.0330, -0.1318,  ...,  0.0681, -0.2237, -0.0329],
         ...,
         [-0.3705, -0.1560,  0.1306,  ..., -0.0414, -0.1212,  0.0239],
         [-0.4461, -0.0353, -0.0569,  ..., -0.3019, -0.2055,  0.1125],
         [-0.7936, -0.3340,  0.4785,  ..., -0.6632,  0.2606, -0.0487]],

        [[ 0.3372, -0.1969, -0.2637,  ...,  0.1285, -0.1950, -0.1072],
         [ 0.1062, -0.0360, -0.0088,  ...,  0.0152,  0.2644, -0.0733],
         [-0.0008, -0.1024, -0.0742,  ..., -0.1044, -0.2028,  0.1422],
         ...,
         [ 0.0888,  0.0863, -0.1041,  ..., -0.3859,  0.1573, -0.2327],
         [-0.6647, -0.2806,  0.4054,  ..., -0.7643,  0.2796, -0.0942],
         [ 0.2810, -0.1080, -0.0029,  ..., -0.0222, -0.1766, -0.1195]],

        [[ 0.2213, -0.2488, -0.1796,  ...,  0.1425, -0.1233, -0.1383],
         [ 0.2064, -0.1551,  0.0255,  ...,  0

### 평가 metric 계산

In [ ]:
def token_accuracy(logits, labels):
  # logits: [batch, seq_len, num_labels]
  # labels; [batch, seq_len]

  # 구현해봅시다!!!!
  # logits에서 가장 높은 확률을 가진 index가 label과 일치하는지 확인해야합니다.
  # 일치하는 값의 비율이 accuracy가 됩니다.
  # 단, labels가 -100인 경우를 제외해야 합니다.

  return acc_sum / acc_count

assert token_accuracy(
    torch.tensor ([
        [0.1, 0.8, 0.1],
        [0.1, 0.8, 0.1],
        [0.2, 0.4, 0.4]
    ]),
    torch.tensor ([
        [
            0,
            1,
            -100
        ]
    ])
) == 0.5 # 정답 0.5

In [ ]:

def token_macro_f1(logits, labels):
  num_labels = logits.shape[-1]
  preds = logits.argmax(-1)

  micro_f1 = []
  total_labels = (labels != -100).sum()

  for i in range(num_labels):
    ### 구현해봅시다!!!! (-100 제외하고 구하기!)
    tp = ?
    fp = ?
    fn = ?

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * (precision * recall) / (precision + recall)

    if not torch.isnan(f1):
      micro_f1.append(f1.item())

  return torch.tensor(micro_f1).mean().item()


assert token_macro_f1(
    torch.tensor ([
        [0.1, 0.8, 0.1],
        [0.1, 0.8, 0.1],
        [0.2, 0.4, 0.4]
    ]),
    torch.tensor ([
        [
            0,
            1,
            -100
        ]
    ])
) == 0.5 # 정답 0.5

# 학습해보기

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

batch_size = 8

filtered_data = tokenized_wnut.map(filter_keys, remove_columns=["id", "tokens", "ner_tags"])
training_loader = torch.utils.data.DataLoader(filtered_data["train"], batch_size=batch_size, shuffle=True, collate_fn=data_collator)
validation_loader = torch.utils.data.DataLoader(filtered_data["validation"], batch_size=batch_size, shuffle=False, collate_fn=data_collator)

In [ ]:
# DataLoader 의 값 살펴보기
next(iter(training_loader))

{'input_ids': tensor([[  101,  2651,  1005,  1055, 15138,  2253,  2013,  9643,  2000, 12476,
          2013,  1996,  2617,  1045,  3651,  1045,  2106,  2025,  2031,  2465,
          2651,  1012,   102,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,  2431, 17776,  1010,  2279,  2129,  2152,  1010, 13055,  3337,
          1010, 23893, 23893,  3413,  1010,  1998,  1018,  2847,  1997,  5958,
          1012,  1004,  8318,  1025,  2757,  2665,  7530,  2015,  1004, 14181,
          1025,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101, 19387,  1030,  6788,  2015, 20554,  1024,  2893,  3201,  2005,
          2256,  2345,  2047,  2265,  1997,  2297,  1012,  6069,  2022,  1037,
          2204,  2028,  1011,  1011,

In [ ]:
from tqdm import tqdm

def change_dict_device(x):
  return {k: v.to(device) for k, v in x.items()}

def train_epoch(epoch_index):
    running_loss, running_acc, running_macro_f1 = 0.0, 0.0, 0.0
    last_loss = 0.
    log_freq = 50

    for i, batch in enumerate(tqdm(training_loader, desc=f"Epoch {epoch_index}")):
        optimizer.zero_grad()

        change_dict_device(batch)

        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]

        logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = token_classification_loss(logits, labels)
        loss.backward()

        optimizer.step()

        running_loss += loss.item()

        if (i + 1) % log_freq == 0:
            last_loss = running_loss / log_freq
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

@torch.no_grad()
def validation_epoch(epoch_index):
    running_loss, running_acc, running_macro_f1 = 0.0, 0.0, 0.0
    total_logits, total_labels = [], []
    for i, batch in enumerate(tqdm(validation_loader, desc=f"Evaluating...", leave=False)):
        change_dict_device(batch)

        input_ids, attention_mask, labels = batch["input_ids"], batch["attention_mask"], batch["labels"]

        logits = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = token_classification_loss(logits, labels)

        running_loss += loss.item()

        total_logits.extend(logits.view(-1, 13).tolist()) # 1차원으로 바꿔서 추가
        total_labels.extend(labels.view(-1).tolist()) # 1차원으로 바꿔서 추가

    running_loss = running_loss / i
    total_logits, total_labels = torch.tensor(total_logits), torch.tensor(total_labels)

    running_acc = token_accuracy(total_logits, total_labels)
    running_macro_f1 = token_macro_f1(total_logits, total_labels)

    print(f"evaluation #{epoch_index}: loss={running_loss}, accuracy={running_acc}, macro-f1={running_macro_f1}")

In [ ]:
EPOCHS = 1

for epoch in range(EPOCHS):
    print('EPOCH {}:'.format(epoch))

    model.train()
    avg_loss = train_epoch(epoch)
    print(f"LOSS train {avg_loss}")

    model.eval()
    validation_epoch(epoch)

EPOCH 0:


Epoch 0:  12%|█▏        | 50/425 [02:19<15:59,  2.56s/it]

  batch 50 loss: 0.46735170535743237


Epoch 0:  24%|██▎       | 100/425 [04:34<14:09,  2.61s/it]

  batch 100 loss: 0.27528622748330234


...

Epoch 0:  94%|█████████▍| 400/425 [18:09<01:11,  2.86s/it]

  batch 400 loss: 0.13688919843174518


Epoch 0: 100%|██████████| 425/425 [19:16<00:00,  2.72s/it]


LOSS train 0.13688919843174518


evaluation #0: loss=0.268198549976602, accuracy=0.9467861652374268, macro-f1=0.2777317762374878


# 실제로 사용해보기


In [ ]:

def predict(text):
  inputs = tokenizer(text, return_tensors="pt", is_split_into_words=True).to(device)
  tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

  logits = model(**inputs)

  preds = logits.argmax(-1).tolist()
  preds = [id2label[i] for i in preds[0]]

  for token, pred in zip(tokens, preds):
    print("prediction: ", token, pred)

predict(tokenized_wnut["validation"][1]["tokens"])

prediction:  [CLS] B-person
prediction:  you O
prediction:  should O
prediction:  ' O
prediction:  ve O
prediction:  stayed O
prediction:  on O
prediction:  red B-location
prediction:  ##ond I-location
prediction:  ##o I-location
prediction:  beach I-location
prediction:  b O
prediction:  ##l O
prediction:  ##vd O
prediction:  . O
prediction:  you O
prediction:  were O
prediction:  in O
prediction:  the O
prediction:  border O
prediction:  ##lines O
prediction:  of O
prediction:  garden B-location
prediction:  ##a I-location
prediction:  / O
prediction:  compton B-location
prediction:  [SEP] B-person
